In [4]:
!pip install pandas
!pip install seaborn
!pip install psycopg2

In [8]:
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import math
import psycopg2

In [15]:
# Return: correlation coefficient
# For a given month in a given region, queries the database to calcualte the correlation coefficient
def query_without_threshold(cursor, region, month):
    
    cursor.execute('''SELECT SUM(temperature) FROM deaths_and_temperature WHERE month = %s AND region = %s;''', (month, region))
    x_sum = cursor.fetchone()[0]

    cursor.execute('''SELECT SUM(totaldeaths) FROM deaths_and_temperature WHERE month = %s AND region = %s;''', (month, region))
    y_sum = cursor.fetchone()[0]

    cursor.execute('''SELECT SUM(temperature * temperature) FROM deaths_and_temperature WHERE month = %s AND region = %s;''', (month, region))
    x_squared_sum = cursor.fetchone()[0]

    cursor.execute('''SELECT SUM(totaldeaths * totaldeaths) FROM deaths_and_temperature WHERE month = %s AND region = %s;''', (month, region))
    y_squared_sum = cursor.fetchone()[0]

    cursor.execute('''SELECT COUNT(*) FROM deaths_and_temperature WHERE month = %s AND region = %s;''', (month, region))
    count_x = cursor.fetchone()[0]

    cursor.execute('''SELECT SUM(temperature * totaldeaths) FROM deaths_and_temperature WHERE month = %s AND region = %s;''', (month, region))
    sum_xy_product = cursor.fetchone()[0]
   
    # math function for the correlation
    try:
        correlation_coefficient = (count_x * sum_xy_product - (x_sum * y_sum)) / math.sqrt((count_x * x_squared_sum - x_sum * x_sum) * (count_x * y_squared_sum - y_sum * y_sum))
    # if denominator is zero, error occurs
    except ZeroDivisionError:
        print("Error: Division by zero. This could be due to the dataset being too small or no data points meeting the threshold.")
        correlation_coefficient = None
    
    return correlation_coefficient

In [24]:
# Return: correlation coefficient
# For a given month in a given region, queries the database to calcualte the correlation coefficient for a spesific threshold for the temperature
def query_with_threshold(cursor, region, month, threshold):
    
    cursor.execute('''SELECT SUM(temperature) FROM deaths_and_temperature WHERE month = %s AND region = %s AND temperature >= %s;''', (month, region, threshold))
    x_sum = cursor.fetchone()[0]

    # Check if count_x is zero
    if x_sum == None:
        return "No data points meet the threshold."
    
    cursor.execute('''SELECT SUM(totaldeaths) FROM deaths_and_temperature WHERE month = %s AND region = %s AND temperature >= %s;''', (month, region, threshold))
    y_sum = cursor.fetchone()[0]
    
    # Check if sum_y is zero
    if y_sum == None:
        return "No data points meet the threshold."
    
    cursor.execute('''SELECT SUM(temperature * temperature) FROM deaths_and_temperature WHERE month = %s AND region = %s AND temperature >= %s;''', (month, region, threshold))
    x_squared_sum = cursor.fetchone()[0]

    cursor.execute('''SELECT SUM(totaldeaths * totaldeaths) FROM deaths_and_temperature WHERE month = %s AND region = %s AND temperature >= %s;''', (month, region, threshold))
    y_squared_sum = cursor.fetchone()[0]

    cursor.execute('''SELECT COUNT(*) FROM deaths_and_temperature WHERE month = %s AND region = %s AND temperature >= %s;''', (month, region, threshold))
    count_x = cursor.fetchone()[0]

    cursor.execute('''SELECT SUM(temperature * totaldeaths) FROM deaths_and_temperature WHERE month = %s AND region = %s AND temperature >= %s;''', (month, region, threshold))
    sum_xy_product = cursor.fetchone()[0]

    # math function for the correlation
    try:
        correlation_coefficient = (count_x * sum_xy_product - (x_sum * y_sum)) / math.sqrt((count_x * x_squared_sum - x_sum * x_sum) * (count_x * y_squared_sum - y_sum * y_sum))
    # if denominator is zero, error occurs
    except ZeroDivisionError:
        print("Error: Division by zero. This could be due to the dataset being too small or no data points meeting the threshold.")
        correlation_coefficient = None

    return correlation_coefficient

In [25]:
# Return: List
# Iterates over all the regions in the dataset and all the months of a year and returns a list with the correlation coefficient for 
# each region and month
def calculate_correlation(threshold):
    # Define PostgreSQL connection parameters
    postgres_params = {
        "host": "127.0.0.1",
        "port": 5439,
        "database": "postgres",
        "user": "test",
        "password": "test",
    }

    # Connect to the postgres databse
    connection = psycopg2.connect(**postgres_params)
    print(connection.get_dsn_parameters(),"\n")

    cursor = connection.cursor()
    cursor.execute('''SELECT DISTINCT region FROM deaths_and_temperature;''')
    regions = cursor.fetchall()

    months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    liste = []
    for region in regions:
        region = region[0]

        for month in months:
            if threshold == None:
                liste.append({'Month': month, 'Region': region, 'Correlation coefficient': query_without_threshold(cursor, region, month), 'Threshold': None})
            else:
                liste.append({'Month': month, 'Region': region, 'Correlation coefficient': query_with_threshold(cursor, region, month, threshold), 'Treshold': threshold})

    # Commit and close the connection
    connection.commit()
    cursor.close()
    connection.close()

    return liste

In [20]:
output_without_threshold = calculate_correlation(None)
for item in output_without_threshold:
    print("Month: {} || Region: {} || Correlation coefficient: {} || Threshold: {}".format(
        item['Month'], item['Region'], item['Correlation coefficient'], item['Threshold']))

{'user': 'test', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '127.0.0.1', 'port': '5439', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

Month: 01 || Region: Paris || Correlation coefficient: -0.14258478268167737 || Threshold: None
Month: 02 || Region: Paris || Correlation coefficient: -0.24130204533734 || Threshold: None
Month: 03 || Region: Paris || Correlation coefficient: 0.007351452216165417 || Threshold: None
Month: 04 || Region: Paris || Correlation coefficient: -0.18014910235395823 || Threshold: None
Month: 05 || Region: Paris || Correlation coefficient: -0.26127321513992063 || Threshold: None
Month: 06 || Region: Paris || Correlation coefficient: -0.07081477108967577 || Threshold: None
Month: 07 || Region: Paris || Correlation coefficient: 0.005752649205437444 || Threshold: None
Month: 08 || Region: Paris || Correla

In [ ]:
# Visualize: heatmap with x-axis: Month and y-axis: correlation coefficient
# Creates a heatmap of the correlation coefficients over the months of the year in different regions
def visualization_question_one():
    # Create a DataFrame from the data
    df = pd.DataFrame(without_threshold, columns=['Month', 'Region', 'Correlation coefficient', 'Threshold'])

    # Drop rows with null 'Correlation coefficient' values
    df = df.dropna(subset=['Correlation coefficient'])
    
    # Convert 'Month' column to numeric
    df['Month'] = pd.to_numeric(df['Month'])

    # Visualize relationship between total deaths and temperature across all regions
    sns.lmplot(x='Month', y='Correlation coefficient', hue='Region', data=df)

    plt.title('Heatmap')
    plt.show()
    
visualization_question_one()

In [ ]:
# Visualize: heatmap with x-axis: Month and y-axis: correlation coefficient
# Creates a heatmap of the correlation coefficients over the months of the year in different regions with threshold
def visualization_question_two():
    # Create a DataFrame from the data
    df = pd.DataFrame(with_threshold, columns=['Month', 'Region', 'Correlation coefficient', 'Threshold'])

    # Drop rows with null 'Correlation coefficient' values
    df = df.dropna(subset=['Correlation coefficient'])
    
    # Convert 'Month' column to numeric
    df['Month'] = pd.to_numeric(df['Month'])

    # Visualize relationship between total deaths and temperature across all regions
    sns.lmplot(x='Month', y='Correlation coefficient', hue='Region', data=df)

    plt.title('Heatmap')
    plt.show()
    
visualization_question_two()